# Downloading & Preprocessing data


In [1]:
# Downloading raw data in data/raw folder
# Data is downloaded from NIH website -> GEO dataset -> Gene Expression Omnibus repository (curated gene expression DataSets
#!bash ../data/download_data.sh 

In [2]:
import sys
path_to_module = '../src/data_preprocess.py'
path_to_module2 = '../src/graph_construction.py'
import os
sys.path.append(os.path.dirname(path_to_module))
sys.path.append(os.path.dirname(path_to_module2))
import data_preprocess,graph_construction
import pandas as pd
from scipy import stats


In [3]:
path_data="../data/raw/data.mtx.gz" 
path_rows="../data/raw/data_rows.txt"
path_cols="../data/raw/data_cols.txt" 
data=data_preprocess.read_rawdata(path_data,0)
#data_rows=data_preprocess.read_rawdata(path_rows,0)
#data_cols=data_preprocess.read_rawdata(path_cols,0)

Shape check [#genes x # cells] : (40614, 3386)


## Preprocess

In [4]:
# Filtering

# Dead cells or bad quality (if expressed genes < (500-100))  (https://www.biorxiv.org/content/10.1101/483297v1.full)
df_filtered=data_preprocess.filter_genes_expressed(data,10000) # chosen 500
#selected 10000 just per afterwards calculation and plots
# Filter non informative genes --> remove genes expressed in very few cells ( < 3)

df_filtered=data_preprocess.filter_genes_not_informative(df_filtered,5)


Cells with <500 expressed genes:  2751
Cells remained after filtering: 635
Genes with < 3 expressed cells: 14234
Filtered dimensions: (26380, 635)


In [5]:
#df=pd.DataFrame   .sparse.from_spmatrix(df_filtered)

In [6]:
#df_norm=data_preprocess.normalize(df,1e4)
df_norm=data_preprocess.log_normalize(df_filtered)

df_reduced=data_preprocess.reduce_dimentionality(df_norm,50,'svd')
df_hvg=data_preprocess.select_highly_variable_genes(df_norm,1000)
'''# Save the dataframes to CSV files
savename="../data/processed/df_reduced.csv"
df_reduced_df = pd.DataFrame(df_reduced)  # Convert to DataFrame
#df_reduced_df.to_csv(savename, index=False)
savename="../data/processed/df_hvg.csv"
df_hvg_df = pd.DataFrame(df_hvg)  # Convert to DataFrame
df_hvg_df.to_csv(savename)'''

[10000. 10000. 10000. 10000. 10000.]
New dimensions:  (635, 50)
Variance explained on first 5 PC: [0.0137539  0.18275619 0.06525957 0.03707839 0.02548461]
Dimensions matrix HVG: (1000, 635)
mean variance of HVG selected: 0.3183693788119099  - mean variance other genes: 0.03592094131700926


'# Save the dataframes to CSV files\nsavename="../data/processed/df_reduced.csv"\ndf_reduced_df = pd.DataFrame(df_reduced)  # Convert to DataFrame\n#df_reduced_df.to_csv(savename, index=False)\nsavename="../data/processed/df_hvg.csv"\ndf_hvg_df = pd.DataFrame(df_hvg)  # Convert to DataFrame\ndf_hvg_df.to_csv(savename)'

## Graph construction

In [7]:
# Nodes: Cells
# Edges: connect cells with similar expression profile
# use Pearson correlation to calculate correlation between cells. Threshold =0.3
# Steps: 1. Matrix correlation, 2. Apply threshold to detect cell connectivity (simlarity). 3. Check diagonal is 0
df=pd.DataFrame(df_reduced)
adj_matrix=graph_construction.adjacent_matrix(df_reduced,0.6)
G_data=graph_construction.createGraph(adj_matrix,df_reduced)


Dimension of matrix correlation: (635, 635)
Threshold 0.6: around 198744 graph connections
Data(x=[635, 50], edge_index=[2, 397488])


In [ ]:
# Visualize graph
import networkx as nx
import matplotlib.pyplot as plt

G= graph_construction.plotGraph(G_data)
